In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

options = webdriver.ChromeOptions()

# Configurar o diretório de download para um local específico
prefs = {'download.default_directory': '/home/douglas/MDS/2023-2-Squad03/diarios'}
options.add_experimental_option('prefs', prefs)

nav = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
nav.get("https://www.diariomunicipal.com.br/ama/")

# Interagir com o site para fazer o download
nav.find_element(By.XPATH, '//*[@id="formCalendario"]/fieldset/div/div[2]/table/tbody/tr[5]/td[4]').click()
elemento = WebDriverWait(nav, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="btDownloadSimples2"]')))

# Obter o nome original do arquivo da URL
url = elemento.get_attribute("href")
nome_original = url.split("publicado_")[-1]
nome_original = "publicado_" + nome_original

# Caminho local completo para o arquivo baixado
caminho_local_download = '/home/douglas/MDS/2023-2-Squad03/diarios' + nome_original

# Clicar para iniciar o download
elemento.click()

# Esperar até que o arquivo seja completamente baixado
caminho_completo = os.path.join('/home/douglas/MDS/2023-2-Squad03/diarios', nome_original)
while not os.path.exists(caminho_completo):
    time.sleep(1)

# Configurar autenticação com o Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Abre uma janela do navegador para autenticação
drive = GoogleDrive(gauth)

# Verificar se o arquivo existe antes de fazer upload
if os.path.exists(caminho_completo):
    # Fazer upload do arquivo renomeado para o Google Drive
    arquivo = drive.CreateFile({'title': 'today.pdf'})  # Nome do arquivo no Google Drive
    arquivo.SetContentFile(caminho_completo)  # Especifique o caminho local do arquivo a ser enviado
    arquivo.Upload()
    print(f'Arquivo {arquivo["title"]} foi enviado para o Google Drive com sucesso.')
else:
    print(f'O arquivo {nome_original} não foi encontrado no caminho especificado.')

# Fechar o navegador após o download e envio do arquivo
nav.quit()
